In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp gdrive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d yelp-dataset/yelp-dataset/yelp_academic_dataset_review.json

100% 4.07G/4.07G [00:45<00:00, 137MB/s]
100% 4.07G/4.07G [00:45<00:00, 96.6MB/s]


In [ ]:
!unzip yelp-dataset.zip -d /content/gdrive/MyDrive/NLP_Project/yelp_dataset

Archive:  yelp-dataset.zip
  inflating: /content/gdrive/MyDrive/NLP_Project/yelp_dataset/Dataset_User_Agreement.pdf  
  inflating: /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_business.json  
  inflating: /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_checkin.json  
  inflating: /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_review.json  
  inflating: /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_tip.json  
  inflating: /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_user.json  


In [ ]:
!ls /content/gdrive/MyDrive/NLP_Project/yelp_dataset/

Dataset_User_Agreement.pdf	     yelp_academic_dataset_review.json
yelp_academic_dataset_business.json  yelp_academic_dataset_tip.json
yelp_academic_dataset_checkin.json   yelp_academic_dataset_user.json


In [ ]:
!rm /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_business.json /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_checkin.json /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_tip.json 

rm: cannot remove '/content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_business.json': No such file or directory


In [ ]:
!rm /content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_user.json

rm: cannot remove '/content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_user.json': No such file or directory
rm: cannot remove '/content/': Is a directory


In [ ]:
import pandas as pd

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

In [ ]:
!ls

gdrive	sample_data  spark-3.0.1-bin-hadoop3.2.tgz


In [ ]:
!tar xf spark-3.0.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
findspark.find()

'/content/spark-3.0.1-bin-hadoop3.2'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import pandas as pd
import numpy as np

In [ ]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [ ]:
review = spark.read.json('/content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_academic_dataset_review.json')

In [ ]:
review.count()

6990280

In [ ]:
review = review.\
withColumn("sentiment_strength",\
           when(col("stars") == 1, "Strong Negative").\
           when(col("stars") == 2, "Negative").\
           when(col("stars") == 3, "Neutral").\
           when(col("stars") == 4, "Positive").\
           when(col("stars") == 5, "Strong Positive").\
           otherwise("No Sentiment")).\
withColumn("label",\
           when(col("stars") == 1, 0).\
#            when(col("stars") == 2, 0).\
#            when(col("stars") == 3, 2).\
#            when(col("stars") == 4, 1).\
           when(col("stars") == 5, 1).\
           otherwise(3))

In [ ]:
review.groupby('label').agg(count('*').alias('record_cnt')).orderBy('label', ascending=False).show()

+-----+----------+
|label|record_cnt|
+-----+----------+
|    3|   2689092|
|    1|   3231627|
|    0|   1069561|
+-----+----------+



In [ ]:
trainingData = review.filter(review['label'] < 3).select('text', 'label')

In [ ]:
ratio_adjust = 1.0 ## ratio of pos to neg in the df_subsample


counts = trainingData.groupBy('label').count().collect()

if counts[0][1] > counts[1][1]:
    down_class = counts[0][0]
else:
    down_class = counts[1][0]
    

higherBound = counts[0][1]
treshold_to_filter = int(ratio_adjust * float(counts[1][1]) / counts[0][1] * higherBound)
 
randGen = lambda x: np.random.randint(0, higherBound) if x == down_class else -1
 
udfRandGen = udf(randGen, IntegerType())

trainingData = trainingData.withColumn("randIndex", udfRandGen("label"))
trainingData = trainingData.filter(trainingData['randIndex'] < treshold_to_filter).drop('randIndex')
 
print("Distribution of Pos and Neg cases of the down-sampled training data are: \n", \
      trainingData.groupBy("label").count().take(3))

Distribution of Pos and Neg cases of the down-sampled training data are: 
 [Row(label=1, count=1069200), Row(label=0, count=1069561)]


In [ ]:
trainingData.groupby('label').agg(count('*').alias('record_cnt')).orderBy('label', ascending=False).show()

+-----+----------+
|label|record_cnt|
+-----+----------+
|    1|   1069454|
|    0|   1069561|
+-----+----------+



In [ ]:
trainingDataSample = trainingData.sample(False, 0.2, 12345)

In [ ]:
trainingDataSample.count()

427355

In [ ]:
pd_reviews = trainingDataSample.toPandas()

In [ ]:
pd_reviews.to_json('/content/gdrive/MyDrive/NLP_Project/yelp_dataset/temp_yelp_review.json', orient='records', lines=True)

In [ ]:
pd_reviews = pd.read_json('/content/gdrive/MyDrive/NLP_Project/yelp_dataset/temp_yelp_review.json', orient='records', lines=True)

In [ ]:
!pip install langdetect
from langdetect import detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.3 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=55ed2fd206d955e3042ced898c84bef5c64164e8380303163bfe08470993d08e
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
def lang_detect(x):
    try:
        return detect(x)
    except:
        return '--'

In [ ]:
%time pd_reviews['lang'] = pd_reviews['text'].apply(lambda x: lang_detect(x))

CPU times: user 41min 39s, sys: 28.2 s, total: 42min 8s
Wall time: 42min 16s


In [ ]:
pd_reviews.head(5)

,text,label,lang
0,Tremendous service (Big shout out to Douglas) ...,1,en
1,"On a scale of one to things that are awesome, ...",1,en
2,Replaced 2 old 70s lennox with 2 new ones. Th...,1,en
3,We were a bit weary about trying the Shellfish...,1,en
4,Awesome little shop. The owner really knows h...,1,en


In [ ]:
pd_reviews = pd_reviews[pd_reviews['lang']=='en'].reset_index(drop=True)
pd_reviews.shape

(427110, 3)

In [ ]:
pd_reviews.to_json('/content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_train_sentiment.json', orient='records', lines=True)

In [ ]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
print(sys.version)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sklearn
import pandas as pd

import wordcloud
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import metrics
import numpy as np

import eli5

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
pd_reviews = pd.read_json('/content/gdrive/MyDrive/NLP_Project/yelp_dataset/yelp_train_sentiment.json', orient='records', lines=True)

In [ ]:
# define X and y
X = pd_reviews['text']
y = pd_reviews['label']

print(f"X Shape: {X.shape}")
print(f"y Shape: {y.shape}")

X Shape: (427110,)
y Shape: (427110,)


In [ ]:
# split X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

print(f"Training records, X_train: {X_train.shape} y_train: {y_train.shape}")
print(f"Testing records, X_test: {X_test.shape} y_test: {y_test.shape}")

Training records, X_train: (320332,) y_train: (320332,)
Testing records, X_test: (106778,) y_test: (106778,)


In [ ]:
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Multinomial NB

In [ ]:
from sklearn.pipeline import make_pipeline
from joblib import dump

In [ ]:
pipe_nb = make_pipeline(
    CountVectorizer(lowercase=False, stop_words='english', ngram_range=(1,3)),
    MultinomialNB()
)

In [ ]:
%time pipe_nb.fit(X_train, y_train)

CPU times: user 2min 53s, sys: 4.35 s, total: 2min 57s
Wall time: 3min 12s


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(lowercase=False, ngram_range=(1, 3),
                                 stop_words='english')),
                ('multinomialnb', MultinomialNB())])

In [ ]:
# make class predictions
%time y_pred = pipe_nb.predict(X_test)

CPU times: user 24.4 s, sys: 123 ms, total: 24.5 s
Wall time: 29.7 s


In [ ]:
# calculate accuracy of class predictions
print(f"Test Accuracy: {metrics.accuracy_score(y_test, y_pred) * 100:.1f}%")

Test Accuracy: 95.2%


In [ ]:
# calculate precision and recall
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95     53577
           1       0.97      0.93      0.95     53201

    accuracy                           0.95    106778
   macro avg       0.95      0.95      0.95    106778
weighted avg       0.95      0.95      0.95    106778



In [ ]:
# calculate the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

[[52144  1433]
 [ 3649 49552]]


In [ ]:
# element = 0
# clf = pipe_nb

# text = X_test.iloc[element]
# prediction = np.where(clf.predict(vect.transform([text])) < 1, "Negative", "Positive").tolist()[element]
# print('Text: >>> ' + text + '\n' + 'Sentiment: >>> ' + prediction)

In [ ]:
%time dump(pipe_nb, "/content/gdrive/MyDrive/NLP_Project/yelp_models/nb.joblib") # dump (Saving it as pipeline)

CPU times: user 2min 44s, sys: 3.19 s, total: 2min 48s
Wall time: 3min 5s


['/content/gdrive/MyDrive/NLP_Project/yelp_models/nb.joblib']

In [ ]:
!ls /content/gdrive/MyDrive/NLP_Project/yelp_models/

nb.joblib


Logistic Regression

In [ ]:
pipe_logreg = make_pipeline(
    CountVectorizer(lowercase=False, stop_words='english', ngram_range=(1,3)),
    LogisticRegression(max_iter=1000)
)

In [ ]:
%time pipe_logreg.fit(X_train, y_train)

CPU times: user 16min 27s, sys: 3min 14s, total: 19min 42s
Wall time: 16min 18s


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(lowercase=False, ngram_range=(1, 3),
                                 stop_words='english')),
                ('logisticregression', LogisticRegression(max_iter=1000))])

In [ ]:
%time y_pred = pipe_logreg.predict(X_test)

CPU times: user 19.9 s, sys: 90.3 ms, total: 20 s
Wall time: 20 s


In [ ]:
print(f"Test Accuracy: {metrics.accuracy_score(y_test, y_pred) * 100:.1f}%")

Test Accuracy: 97.8%


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     53577
           1       0.98      0.98      0.98     53201

    accuracy                           0.98    106778
   macro avg       0.98      0.98      0.98    106778
weighted avg       0.98      0.98      0.98    106778



In [ ]:
%time dump(pipe_logreg, "/content/gdrive/MyDrive/NLP_Project/yelp_models/logreg.joblib")

CPU times: user 2min 21s, sys: 2.7 s, total: 2min 24s
Wall time: 2min 27s


['/content/gdrive/MyDrive/NLP_Project/yelp_models/logreg.joblib']

SVM

In [ ]:
pipe_svm = make_pipeline(
    CountVectorizer(lowercase=False, stop_words='english', ngram_range=(1,3)),
    SGDClassifier(max_iter=100, tol=None)
)

In [ ]:
%time pipe_svm.fit(X_train, y_train)

CPU times: user 4min 6s, sys: 4.31 s, total: 4min 10s
Wall time: 4min 9s


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(lowercase=False, ngram_range=(1, 3),
                                 stop_words='english')),
                ('sgdclassifier', SGDClassifier(max_iter=100, tol=None))])

In [ ]:
y_pred = pipe_svm.predict(X_test)

In [ ]:
# calculate accuracy of class predictions
print(metrics.accuracy_score(y_test, y_pred))

0.9782914083425425


In [ ]:
# calculate precision and recall
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     53577
           1       0.98      0.98      0.98     53201

    accuracy                           0.98    106778
   macro avg       0.98      0.98      0.98    106778
weighted avg       0.98      0.98      0.98    106778



In [ ]:
%time dump(pipe_svm, "/content/gdrive/MyDrive/NLP_Project/yelp_models/svm.joblib")

CPU times: user 2min 21s, sys: 2.72 s, total: 2min 24s
Wall time: 2min 28s


['/content/gdrive/MyDrive/NLP_Project/yelp_models/svm.joblib']